# Purpose

This notebook demonstrates the data pipeline from raw tables to analytical datasets. At the end of this activity, train & test data sets are created from raw data.

## Imports

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from ta_lib.core.api import get_package_path
PACKAGE_PATH=get_package_path()

In [0]:
import sys
import os.path as op
CURR_LOC_IN_DBFS=op.join(PACKAGE_PATH,"..","databricks/notebooks/03_reference_notebooks/python/")
sys.path.append(CURR_LOC_IN_DBFS)

In [0]:
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.options.mode.use_inf_as_na = True

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [0]:
import warnings

warnings.filterwarnings('ignore', message="The sklearn.metrics.classification module", category=FutureWarning)
warnings.filterwarnings('ignore', message=".*title_format is deprecated. Please use title instead.*")
warnings.filterwarnings('ignore', message="optional dependency `torch` is not available. - skipping import of NN models.")

In [0]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer,
    display_as_tabs, string_cleaning, merge_info, initialize_environment
)
import ta_lib.core.api as dataset
import ta_lib.eda.api as ta_analysis

os.environ['TA_DEBUG'] = "False"
os.environ['TA_ALLOW_EXCEPTIONS'] = "True"

# Initialization
initialize_environment(debug=False, hide_warnings=True)

In [0]:
REPORT_LOC = op.join(PACKAGE_PATH,'..','databricks')

## Utility functions

In [0]:
config_path = op.join(CURR_LOC_IN_DBFS,'conf', 'config.yml')
context = create_context(config_path)
print(dataset.list_datasets(context))

## Load and clean data

In [0]:
# Loading raw datasets in a loop
data = dict()
dict_area_code_dup = {
    'BIHAE': 'BSHAE',
    'BIHAW': 'BSHAW',
    'COIM': 'COSM',
    'DELHM': 'DEQHM',
    'HUBL': 'HUBQ',
    'KOLK': 'KOQH',
    'ORIS': 'ORSS',
    'VIJA': 'VSJA',
    }

for i in dataset.list_datasets(context):
    if '/raw/' in i:
        dataset_name = i.replace('/raw/','')
        key_ = dataset_name+'_df'
        data[key_] = dataset.load_dataset(context,i)
        
        # Standardize column names
        data[key_].columns = string_cleaning(data[key_].columns,lower=True)
        
        # Data type changes
        if key_ == 'doj_df':
            data[key_]['date_of_joining'] = pd.to_datetime(data[key_]['date_of_joining'])
        elif key_ == 'returns_df':
            data[key_] = data[key_].replace('#NAME?',np.nan)
            data[key_] = data[key_].replace('inf',np.nan)
            for float_col in ['profit_without_udaan_sub','profit_with_udaan_without_sub',
                              'profit_without_udaan_with_sub','profit_with_udaan_with_sub',
                              'numerator_without_udaan_sub','numerator_with_udaan_without_sub',
                              'numerator_without_udaan_with_sub','numerator_with_udaan_with_sub',
                              'roi_without_udaan_sub','roi_with_udaan_without_sub','roi_without_udaan_with_sub',
                              'roi_with_udaan_with_sub']:
                data[key_][float_col] = data[key_][float_col].astype(float)
        
        # Deduplicating area codes
        if 'asm_area_code' in data[key_].columns:
            fil_ = data[key_]['asm_area_code'].isin(dict_area_code_dup.keys())
            data[key_].loc[fil_,'asm_area_code'] = data[key_].loc[fil_,'asm_area_code'].map(dict_area_code_dup)
        
        # adding start date of the month as a column in tables with year and month.
        if ('year' in data[key_].columns) and ('month' in data[key_].columns):
            data[key_]['month_start_date'] = pd.to_datetime(dict(year=data[key_].year, month=data[key_].month, day=[1]*len(data[key_])))
        
        # Drop duplicates
        data[key_].drop_duplicates(inplace=True)
        
        # Save processed data
        dataset.save_dataset(context, data[key_], 'cleaned/'+dataset_name)

## Define DV

Identification of Customer churn can be done in many ways. Some frequent ways of identification are as follows
 - Case 1 : A customer with no transactions/sales post a `ref date` is considered to be attrited post the said `ref date`.
 - Case 2 : In cases where the customer doesn't need to check-in or to do business every month or has a surrender date, churn cannot be defined directly. In such cases, the customer purchase frequency pattern can be monitored to identify attrition. For example, if 95th percentile value of any consecutive

Evaluating which of the above 2 cases does the data belong to -

In [0]:
a = data['pri_bpm_df'].query('pri_sales_amount < 0').groupby('customer_code').agg({'month_start_date':['min','max']})
b = data['pri_bpm_df'].query('pri_sales_amount > 0').groupby('customer_code').agg({'month_start_date':['min','max']})
a.columns = ['neg_min','neg_max']
b.columns = ['pos_min','pos_max']
c = a.merge(b, left_index=True, right_index=True)
print(c.shape, sum(c['neg_min']>c['pos_max']))

This implies that in all the customers once the sales turn negative , they dont turn positive again. Thus inferring case 1

Reference Date (assumed and considered) :: 2013-05-01

In [0]:
ref_date_for_churn = r'2013-05-01'

#### Customer Population

Exhaustive set of customer ids in the data to be considered for churn

 - Date range of the primary sales data is 2011-04 till 2013-12. Exhaustive list of customers prone to churn can be considered as the ones that have been active during 2011-04 till 2013-12.
 - Also only those customers whose date_of_joining is before the reference date are to be considered.

In [0]:
df_population = data['pri_bpm_df'][['customer_code']].drop_duplicates() \
                    .merge(data['doj_df'], on='customer_code', how='left', validate='1:1') \
                    .query(f"date_of_joining <= '{ref_date_for_churn}'or date_of_joining.isnull()",engine='python')[['customer_code']]
df_population.drop_duplicates(inplace=True)

#### Ref-Date Based Decision Variable Generation for Churn.
 - No sales post a date is considered customer churned.
 - Ref date is used to distinguish between customer about to churn in the future to the customers that have already churned in the past.

In [0]:
# obtaining the min month start date when the sales become 0 or negative and considering that month as attrition month
cust = data['pri_bpm_df'] \
           .query('pri_sales_amount <= 0') \
           .groupby('customer_code')['month_start_date'].min() \
           .rename('attrition_month_strt') \
           .reset_index()

cust['ref_date'] = pd.to_datetime(ref_date_for_churn)

# obtaining attrition month - ref month days
cust['days_from_ref_date_to_attrition'] = (cust.attrition_month_strt - cust.ref_date).dt.days
cust['target'] = 1
df_population = df_population.merge(cust, on='customer_code', how='left')

# filtering out customers who have already been churned before the reference date
df_population = df_population[(df_population.days_from_ref_date_to_attrition>=0) | (df_population.days_from_ref_date_to_attrition.isnull())]

df_population = df_population.drop(['ref_date', 'days_from_ref_date_to_attrition', 'attrition_month_strt'], axis=1)
df_population['target'].fillna(0, inplace=True)
df_population['ref_date']=pd.to_datetime(ref_date_for_churn)

print('target summary',df_population.target.describe())

In [0]:
displayHTML(display_as_tabs([('head',df_population.head()), ('tail',df_population.tail())], cloud_env='Databricks'))

### Basic Features

Most of the features here are derived from the data sources and are to be aggregated to be at customer level. Basic customer level features are added here. Rest are generated as part of feature generation

### Customer area code

In [0]:
df_customer_area_code = pd.DataFrame()
for key_ in ['pri_bpm_df','coverage_df','retail_program_df','ec_df',
             'ordered_with_app_df','ordered_without_app_df','dist_retail_invoice_df']:
    df_customer_area_code = df_customer_area_code.append(
        data[key_]
        .filter(['customer_code','asm_area_code'])
        .drop_duplicates()
    ).drop_duplicates()
    
df_population = df_population.merge(
    df_customer_area_code,
    on = 'customer_code',
    how = 'left',
    validate = '1:1'
)

### Date joined

In [0]:
df_population = df_population.merge(
    data['doj_df'],
    on = 'customer_code',
    how = 'left',
    validate = '1:1'
)

**DEV Notes**

<details>
    
    1. validate='1:1' : This ensures extra rows will not be created even by accident.
    2. For more merge rules refer guidelines [here]https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html)
    3. More information on a merge can be obtained by using the ta_lib.core.api.merge_info function

## Test Train Split.

In [0]:
from sklearn.model_selection import train_test_split

df_population.set_index('customer_code',inplace=True)
train_X, test_X, train_y, test_y = train_test_split(
    df_population.drop('target',axis=1), df_population[['target']], test_size=0.2, random_state=context.random_seed)

train_X = train_X.reset_index()
train_y = train_y.reset_index()
test_X = test_X.reset_index()
test_y = test_y.reset_index()

print(f"Saving training datasets")
dataset.save_dataset(context, train_X, 'train/attrition/features')
dataset.save_dataset(context, train_y, 'train/attrition/target')

print(f"Saving test datasets")
dataset.save_dataset(context, test_X, 'test/attrition/features')
dataset.save_dataset(context, test_y, 'test/attrition/target')